In [2]:
import pandas as pd
import numpy as np

In [3]:
ratings = pd.read_csv("ml-latest-small/ratings.csv")
n_rows = ratings.shape[0]

print("Number of rows: " + str(n_rows))
# Show the first 10 rows of the dataframe
print(ratings.head(10).to_string(index=False))

Number of rows: 100836
 userId  movieId  rating  timestamp
      1        1     4.0  964982703
      1        3     4.0  964981247
      1        6     4.0  964982224
      1       47     5.0  964983815
      1       50     5.0  964982931
      1       70     3.0  964982400
      1      101     5.0  964980868
      1      110     4.0  964982176
      1      151     5.0  964984041
      1      157     5.0  964984100


In [4]:
movies = pd.read_csv("ml-latest-small/movies.csv")
n_rows = movies.shape[0]

print("Number of rows: " + str(n_rows))
# Show the first 10 rows of the dataframe
print(movies.head(10).to_string(index=False))

Number of rows: 9742
 movieId                              title                                      genres
       1                   Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
       2                     Jumanji (1995)                  Adventure|Children|Fantasy
       3            Grumpier Old Men (1995)                              Comedy|Romance
       4           Waiting to Exhale (1995)                        Comedy|Drama|Romance
       5 Father of the Bride Part II (1995)                                      Comedy
       6                        Heat (1995)                       Action|Crime|Thriller
       7                     Sabrina (1995)                              Comedy|Romance
       8                Tom and Huck (1995)                          Adventure|Children
       9                Sudden Death (1995)                                      Action
      10                   GoldenEye (1995)                   Action|Adventure|Thriller


In [5]:
def create_user_movie_rating_matrix():
    """
    Creates a user-movie rating matrix.

    Returns:
        DataFrame: A matrix where rows represent users, columns represent movies,
                   and each cell contains the rating given by the user to the movie.
    """
    # Create a pivot table with user ratings for each movie
    user_movie_ratings_matrix = pd.pivot_table(ratings, values='rating', index='userId', columns='movieId')
    # Get unique movie IDs
    all_movie_ids = movies['movieId'].unique()
    # Reindex the matrix columns with all movie IDs
    user_movie_ratings_matrix = user_movie_ratings_matrix.reindex(columns=all_movie_ids)
    return user_movie_ratings_matrix  # Return the user-movie rating matrix

In [6]:
def get_ratings_dict():
    """
    Creates a dictionary to store movie ratings for each user.

    Returns:
        dict: A dictionary containing movie ratings for each user.
              Keys are user IDs, and values are lists of tuples (movieId, rating).
    """
    # Dictionary to store the pairs (movieId,ratings) for each user
    user_data = {}
    # Populate the dictionary
    for _, row in ratings.iterrows():
        userId = row['userId']
        movieId = int(row['movieId'])
        rating = row['rating']
        # Check if userId is already present in the dictionary
        if userId in user_data:
            user_data[userId].append((movieId, rating))
        else:
            # If userId is not present, create a new list with the tuple (movieId, rating)
            user_data[userId] = [(movieId, rating)]
    
    return user_data

In [7]:
def pearsonCorrelation(user1, user2, user_data):
    """
    Calculates the Pearson correlation coefficient between two users based on their ratings.

    Args:
        user1 (int): The ID of the first user.
        user2 (int): The ID of the second user.
        user_data (dict): A dictionary containing movieId-ratings pairs for each user.

    Returns:
        float: The Pearson correlation coefficient between the two users.
    """
    num, den1, den2 = 0.0, 0.0, 0.0
    
    # Fetch ratings of user1 and calculate mean rating of user1
    ratings_user1 = user_data[user1]
    user1_ratings = np.array([rating for _, rating in ratings_user1])
    rmean_user1 = np.mean(user1_ratings)
    
    # Fetch ratings of user2 and calculate mean rating of user2
    ratings_user2 = user_data[user2]
    user2_ratings = np.array([rating for _, rating in ratings_user2])
    rmean_user2 = np.mean(user2_ratings)

    # set of movieIds evaluated by user1
    items_user1 = {movieId for movieId, _ in ratings_user1}
    # set of movieIds evaluated by user2
    items_user2 = {movieId for movieId, _ in ratings_user2}
    # set of movieIds evaluated by both user1 and user2 (intersection)
    common_items = items_user1 & items_user2

    # Calculate Pearson correlation for common items
    for p in common_items:
        # Fetch ratings of user1 and user2 for the common item
        r_1p = next(r for m, r in ratings_user1 if m == p)
        r_2p = next(r for m, r in ratings_user2 if m == p)
        num += (r_1p - rmean_user1) * (r_2p - rmean_user2)
        den1 += (r_1p - rmean_user1) ** 2
        den2 += (r_2p - rmean_user2) ** 2
    
    # Calculate denominator
    den = np.sqrt(den1 * den2) if den1 != 0 and den2 != 0 else 0.0
    
    # Check for division by zero
    if den == 0.0:
        return 0.0
    
    # Calculate Pearson correlation coefficient
    sim = num / den
    return sim


In [8]:
def compute_all_user_similarities():
    """
    Computes similarities between all pairs of users based on their movie ratings.

    Returns:
        dict: A dictionary of dictionaries containing similarities between all pairs of users.
              Keys are user IDs, and values are dictionaries where keys are other user IDs
              and values are similarity scores.
    """
    num_users = ratings["userId"].nunique()
    similarities_dict = {}
    # Get movie ratings for each user
    user_data = get_ratings_dict()
    # Iterate through all user pairs
    for user1 in range(1,num_users+1):
        similarities_dict[user1] = {}
        for user2 in range(user1,num_users+1):
            if not user2 in similarities_dict:
                similarities_dict[user2] = {}
            if user1 != user2:
                # Calculate similarity between user1 and user2
                sim = pearsonCorrelation(user1,user2, user_data)
                # Store similarity for both user1 and user2 in the dictionary
                similarities_dict[user1][user2] = sim
                similarities_dict[user2][user1] = sim
    return similarities_dict 

In [9]:
def generatePrediction(user, item, user_movie_ratings_matrix, similarities_dict, rmean_user, topK_simlar_users):
    """
    Generates a prediction for a user's rating on a specific item (movie) based on collaborative filtering.

    Args:
        user (int): The ID of the user.
        item (int): The ID of the item (movie).
        user_movie_ratings_matrix (DataFrame): A DataFrame containing user-item ratings.
        similarities_dict (dict): A dictionary containing similarities between users.
        rmean_user (float): The mean rating of the user.
        topK_similar_users (list): A list of IDs of top K similar users to the target user.

    Returns:
        float: The predicted rating for the user on the specified item.
    """
    # Check if the movie has already been rated by the user
    if not np.isnan(user_movie_ratings_matrix.at[user, item]):
        return user_movie_ratings_matrix.at[user, item]
    
    num, den = 0.0, 0.0
    # Iterate through top K similar users
    for u in topK_simlar_users:
        if not np.isnan(user_movie_ratings_matrix.at[u, item]):        
            rmean_u = user_movie_ratings_matrix.loc[u].mean()   # Mean rating of the similar user u
            r_up = user_movie_ratings_matrix.at[u, item]    # Rating of item by user u
            similarity = similarities_dict[user][u]     # Similarity between target user and user u
            num += similarity * (r_up - rmean_u)
            den += abs(similarity)
    # Calculate the predicted rating
    if den == 0.0:
        # Use the mean rating of the target user if no similar users have rated the item
        pred = rmean_user
    else:
        # Calculate the prediction using the collaborative filtering formula
        pred = rmean_user + (num / den)
    return pred

In [10]:
def get_unrated_movie_ids(userId):
    """
    Finds the movie IDs that have not been rated by the specified user.

    Args:
        userId (int): The ID of the user.

    Returns:
        list: A list of movie IDs that have not been rated by the specified user.
    """
    # Get all unique movie IDs present in the ratings DataFrame
    all_movie_ids = movies['movieId'].tolist()
    # Get the movie IDs rated by the specified userId
    rated_movie_ids = ratings[ratings['userId'] == userId]['movieId'].tolist()
    # Find the movie IDs not rated by the userId
    unrated_movie_ids = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movie_ids]
    return unrated_movie_ids

# a function that, given a list of users as input, returns the union of unrated movies for each user.
def get_all_unrated_movies(group):
    unrated_movies = set()
    # for each user call the function getUnratedMovies, which returns the movies not rated by that user
    for user in group:
        user_unrated_movies = get_unrated_movie_ids(user)
        #and update the list to store all of them
        unrated_movies.update(user_unrated_movies)
    return list(unrated_movies)

In [11]:
def kMostSimilarUsers(user, k, similarities_dict):
    """
    Gets the top k most similar users to a given user based on similarity coefficients.

    Args:
        user (int): The ID of the target user.
        k (int): The number of most similar users to retrieve.
        similarities_dict (dict): A dictionary containing similarities between users.

    Returns:
        list: A list of IDs of the top k most similar users to the target user.
    """
    # Get similarities of the target user with other users
    user_similarities = similarities_dict[user]
    # Sort users by similarity coefficient in descending order
    sorted_user_similarities = sorted(user_similarities, key=user_similarities.get, reverse=True)
    # Extract the top k similar users and their corresponding similarity coefficients
    top_k_users = sorted_user_similarities[:k]
    return top_k_users

# Assignment 3

### Design and implement a new method for producing sequential group recommendations. Also, provide detailed explanations and clarifications about why the method you propose works well for the case of sequential group recommendations. Hint: There is no need to design a method from scratch. For the needs of this assignment, you can suggest simple modifications of the existing approach, e.g., by proposing and using alternatives for group aggregation that ensure good results for the group.

In [12]:
def generatePredictionOnUnratedMovies(user, unrated_movies, ratings_matrix, similarities_dict):
    """
    Generates predictions for unrated movies for a given user.

    Args:
        user (int): The ID of the user.
        unrated_movies (list): A list of unrated movie IDs for the user.
        ratings_matrix (DataFrame): A DataFrame containing user-item ratings.
        similarities_dict (dict): A dictionary containing similarities between users.

    Returns:
        list: A list of tuples containing the predicted ratings for unrated movies.
              Each tuple contains the movie ID and its predicted rating.
    """
    predictions = []
    # Get top 50 similar users for the target user
    topK_simlar_users = kMostSimilarUsers(user, 50, similarities_dict)
    # Calculate the average of the user's ratings
    rmean_user = ratings_matrix.loc[user].mean()
    # Generate predictions for each unrated movie
    for item in unrated_movies:
        pred = generatePrediction(user, item, ratings_matrix, similarities_dict, rmean_user, topK_simlar_users)
        predictions.append((item, pred))
    return predictions

In [13]:
def topKMoviesHybrid(k, alpha, predictions, excluded_movies):
    """
    Generates top-k hybrid movie recommendations based on a combination of individual predictions.

    Args:
        k (int): The number of top recommendations to generate.
        alpha (float): The weight parameter for balancing between average and minimum ratings.
        predictions (list): A list of predictions for each movie.
                           Each prediction is a tuple (movie_id, predicted_rating, individual_ratings).
        excluded_movies (list): A list of movie IDs to be excluded from recommendations.

    Returns:
        list: A list of tuples containing the top-k hybrid movie recommendations.
              Each tuple contains the movie ID, hybrid rating, and individual ratings.
    """
    # List to store the hybrid predictions
    hybrid_predictions = []

    # Iterate over each movie
    for i in range(len(predictions[0])):
        # Calculate the average rating for each movie
        avg_rating = sum(prediction[i][1] for prediction in predictions) / len(predictions)
        
        # Calculate the minimum rating for each movie
        min_rating = min(prediction[i][1] for prediction in predictions)

        # Build the list of tuples (movie_id, hybrid_rating, individual_ratings)
        movie_id = predictions[0][i][0]
        individual_ratings = [prediction[i][1] for prediction in predictions]
        # Check if the movie is not in the excluded list
        if movie_id not in excluded_movies:
            # Calculate the hybrid rating using the weighted combination of average and minimum ratings
            hybrid_rating = (1 - alpha) * avg_rating + alpha * min_rating
            hybrid_predictions.append((movie_id, hybrid_rating, individual_ratings))
    
    # Ensure that k is not greater than the number of available recommendations
    if(len(hybrid_predictions) < k):
        k = len(hybrid_predictions)

    # Sort the list based on the hybrid rating (in descending order)
    sorted_hybrid_predictions = sorted(hybrid_predictions, key=lambda x: x[1], reverse=True)
    
    # Take only the first k elements from the sorted list
    top_k_hybrid_predictions = sorted_hybrid_predictions[:k]
    
    return top_k_hybrid_predictions

In [14]:
def computeUsersSatisfaction(group, group_top_k, user_top_k):
    """
    Computes the satisfaction of users in a group based on their individual and group top-k movie recommendations.

    Args:
        group (list): A list of user IDs forming the group.
        group_top_k (list): A list of top-k hybrid movie recommendations for the group.
                            Each recommendation is a tuple (movie_id, hybrid_rating, individual_ratings).
        user_top_k (list): A list of top-k hybrid movie recommendations for individual users.
                           Each recommendation is a tuple (movie_id, hybrid_rating, individual_ratings).

    Returns:
        list: A list of user satisfactions based on their individual and group recommendations.
    """
    # Calculate the total satisfaction for each user in the group
    usersListSat = [sum(rating for _, rating in elem) for elem in user_top_k]
    # Calculate the total satisfaction for the group
    groupListSat = list(map(sum, zip(*[elem[2] for elem in group_top_k])))
    satisfactions = []
    # Compute satisfaction for each user in the group
    for i in range(0,len(group)):
        num = groupListSat[i]
        den = usersListSat[i]
        sat = num/den
        satisfactions.append(sat)
    return satisfactions  

In [15]:
def topKMoviesHybridWithSTD(k, alpha, predictions, excluded_movies):
    """
    Generates top-k hybrid movie recommendations based on a combination of average rating and standard deviation.

    Args:
        k (int): The number of top recommendations to generate.
        alpha (float): The weight parameter for balancing between average rating and standard deviation.
        predictions (list): A list of predictions for each movie.
                           Each prediction is a tuple (movie_id, predicted_rating, individual_ratings).
        excluded_movies (list): A list of movie IDs to be excluded from recommendations.

    Returns:
        list: A list of tuples containing the top-k hybrid movie recommendations.
              Each tuple contains the movie ID, hybrid rating, and individual ratings.
    """
    # List to store the hybrid predictions
    hybrid_predictions = []

    # Iterate over each movie
    for i in range(len(predictions[0])):
        # Calculate the average rating for each movie
        avg_rating = sum(prediction[i][1] for prediction in predictions) / len(predictions)
        
        # Calculate the standard deviation of ratings for each movie
        ratings = [prediction[i][1] for prediction in predictions]
        std_dev_rating = np.std(ratings)

        # Build the list of tuples (movie_id, hybrid_rating, individual_ratings)
        movie_id = predictions[0][i][0]
        individual_ratings = [prediction[i][1] for prediction in predictions]
        if movie_id not in excluded_movies:
            hybrid_rating = (1 - alpha) * avg_rating + alpha * std_dev_rating
            hybrid_predictions.append((movie_id, hybrid_rating, individual_ratings))
    
    if(len(hybrid_predictions) < k):
        k = len(hybrid_predictions)

    # Sort the list based on the hybrid rating (in descending order)
    sorted_hybrid_predictions = sorted(hybrid_predictions, key=lambda x: x[1], reverse=True)
    
    # Take only the first k elements from the sorted list
    top_k_hybrid_predictions = sorted_hybrid_predictions[:k]
    
    return top_k_hybrid_predictions


In [16]:
def sequentialGroupPrediction(group, k, n_iter):
    """
    Performs sequential group prediction iterations to recommend movies to a group of users.

    Args:
        group (list): A list of user IDs forming the group.
        k (int): The number of top recommendations to generate for each iteration.
        n_iter (int): The number of iterations to perform.

    Returns:
        None
    """
    similarities_dict = compute_all_user_similarities()     # Compute similarities between all users
    unrated_movies = get_all_unrated_movies(group)      # Get unrated movies for the group
    ratings_matrix = create_user_movie_rating_matrix()      # Create user-movie rating matrix
    predictions = []    # List to store predictions for each user in the group
    user_top_k = []   # List to store top-k recommendations for each user in the group
    # Generate predictions and top-k recommendations for each user in the group
    for user in group:
        user_predictions = generatePredictionOnUnratedMovies(user, unrated_movies,ratings_matrix,similarities_dict)
        predictions.append(user_predictions)
        sorted_predictions = sorted(user_predictions, key=lambda x: x[1], reverse=True)
        top_k = sorted_predictions[:k]
        user_top_k.append(top_k)
        
    alpha = 0
    excluded_movies = []
    # List to store top-k hybrid movie recommendations for the group
    group_top_k = [] 
    # Perform sequential iterations
    for i in range(0,n_iter):
        # Generate top-k hybrid movie recommendations for the group
        group_top_k = topKMoviesHybrid(k, alpha, predictions, excluded_movies)
        # Compute user satisfactions based on individual and group recommendations
        users_satisfactions = computeUsersSatisfaction(group, group_top_k, user_top_k)
        print("Iteration:",i+1,", alpha =",alpha)
        print(f"The top {k} movies recommended for the group {group} are:")
        # Print top-k recommendations for the group
        for idx, (movie_id, pred, _) in enumerate(group_top_k, 1):
            movie_title = movies.loc[movies['movieId'] == movie_id, 'title'].values[0]
            print(f"{idx}. MovieID: {movie_id}, Title: {movie_title}, Score: {pred}")
            excluded_movies.append(movie_id)
        
        # Print user satisfactions
        for idx, satisfaction in enumerate(users_satisfactions):
            print(f"User {group[idx]}'s satisfaction: ",satisfaction)
        print("Group's satisfaction: ",sum(users_satisfactions)/len(group))
        print("-------------------------------------------------------------------")
        # Update alpha for the next iteration
        alpha = max(users_satisfactions) - min(users_satisfactions)

In [17]:
def sequentialGroupPredictionWithSTD(group, k, n_iter):
    """
    Performs sequential group prediction iterations using standard deviation in recommendations.

    Args:
        group (list): A list of user IDs forming the group.
        k (int): The number of top recommendations to generate for each iteration.
        n_iter (int): The number of iterations to perform.

    Returns:
        None
    """
    similarities_dict = compute_all_user_similarities()     # Compute similarities between all users
    unrated_movies = get_all_unrated_movies(group)      # Get unrated movies for the group
    ratings_matrix = create_user_movie_rating_matrix()      # Create user-movie rating matrix
    predictions = []    # List to store predictions for each user in the group
    user_top_k = []   # List to store top-k recommendations for each user in the group
    # Generate predictions and top-k recommendations for each user in the group
    for user in group:
        user_predictions = generatePredictionOnUnratedMovies(user, unrated_movies,ratings_matrix,similarities_dict)
        predictions.append(user_predictions)
        sorted_predictions = sorted(user_predictions, key=lambda x: x[1], reverse=True)
        top_k = sorted_predictions[:k]
        user_top_k.append(top_k)
        
    alpha = 0
    excluded_movies = []
    # List to store top-k hybrid movie recommendations for the group
    group_top_k = [] 
    # Perform sequential iterationsper il gruppo
    group_top_k = [] 
    for i in range(0,n_iter):
        # Generate top-k hybrid movie recommendations for the group using standard deviation
        group_top_k = topKMoviesHybridWithSTD(k, alpha, predictions, excluded_movies)
        # Compute user satisfactions based on individual and group recommendations
        users_satisfactions = computeUsersSatisfaction(group, group_top_k, user_top_k)
        print("Iteration:",i+1,", alpha =",alpha)
        print(f"The top {k} movies recommended for the group {group} are:")
        # Print top-k recommendations for the group
        for idx, (movie_id, pred, _) in enumerate(group_top_k, 1):
            movie_title = movies.loc[movies['movieId'] == movie_id, 'title'].values[0]
            print(f"{idx}. MovieID: {movie_id}, Title: {movie_title}, Score: {pred}")
            excluded_movies.append(movie_id)
        
        # Print user satisfactions
        for idx, satisfaction in enumerate(users_satisfactions):
            print(f"User {group[idx]}'s satisfaction: ",satisfaction)
        print("Group's satisfaction: ",sum(users_satisfactions)/len(group))
        
        print("-------------------------------------------------------------------")
        alpha = max(users_satisfactions) - min(users_satisfactions)

# Experiments

### Produce a group of 3 users, and for this group, show the top-10 recommendations in 3 different sequences, i.e., the 10 movies with the highest prediction scores in 3 rounds, using the MovieLens 100K rating dataset.

In [17]:
group = [59,125,471]
sequentialGroupPrediction(group,10,3)

Iteration: 1 , alpha = 0
The top 10 movies recommended for the group [59, 125, 471] are:
1. MovieID: 1327, Title: Amityville Horror, The (1979), Score: 5.791721570373908
2. MovieID: 2700, Title: South Park: Bigger, Longer and Uncut (1999), Score: 5.7140822560550015
3. MovieID: 1350, Title: Omen, The (1976), Score: 5.6638891536663545
4. MovieID: 2513, Title: Pet Sematary (1989), Score: 5.605384360481889
5. MovieID: 1248, Title: Touch of Evil (1958), Score: 5.457675224386862
6. MovieID: 1231, Title: Right Stuff, The (1983), Score: 5.438324800273521
7. MovieID: 3972, Title: Legend of Drunken Master, The (Jui kuen II) (1994), Score: 5.393851867385784
8. MovieID: 1924, Title: Plan 9 from Outer Space (1959), Score: 5.355094945391147
9. MovieID: 2583, Title: Cookie's Fortune (1999), Score: 5.218396254855086
10. MovieID: 3543, Title: Diner (1982), Score: 5.165398261006952
User 59's satisfaction:  0.9113830743121526
User 125's satisfaction:  0.9273836153306404
User 471's satisfaction:  0.806434

In [18]:
group = [59,125,471]
sequentialGroupPredictionWithSTD(group,10,3)

Iteration: 1 , alpha = 0
The top 10 movies recommended for the group [59, 125, 471] are:
1. MovieID: 1327, Title: Amityville Horror, The (1979), Score: 5.791721570373908
2. MovieID: 2700, Title: South Park: Bigger, Longer and Uncut (1999), Score: 5.7140822560550015
3. MovieID: 1350, Title: Omen, The (1976), Score: 5.6638891536663545
4. MovieID: 2513, Title: Pet Sematary (1989), Score: 5.605384360481889
5. MovieID: 1248, Title: Touch of Evil (1958), Score: 5.457675224386862
6. MovieID: 1231, Title: Right Stuff, The (1983), Score: 5.438324800273521
7. MovieID: 3972, Title: Legend of Drunken Master, The (Jui kuen II) (1994), Score: 5.393851867385784
8. MovieID: 1924, Title: Plan 9 from Outer Space (1959), Score: 5.355094945391147
9. MovieID: 2583, Title: Cookie's Fortune (1999), Score: 5.218396254855086
10. MovieID: 3543, Title: Diner (1982), Score: 5.165398261006952
User 59's satisfaction:  0.9113830743121526
User 125's satisfaction:  0.9273836153306404
User 471's satisfaction:  0.806434

### 3 most similar users

In [18]:
group = [8,36,44]
sequentialGroupPrediction(group,10,3)

Iteration: 1 , alpha = 0
The top 10 movies recommended for the group [8, 36, 44] are:
1. MovieID: 177593, Title: Three Billboards Outside Ebbing, Missouri (2017), Score: 4.563391071103838
2. MovieID: 174053, Title: Black Mirror: White Christmas (2014), Score: 4.540030783592607
3. MovieID: 1207, Title: To Kill a Mockingbird (1962), Score: 4.451569403617054
4. MovieID: 2359, Title: Waking Ned Devine (a.k.a. Waking Ned) (1998), Score: 4.431767139479905
5. MovieID: 3347, Title: Never Cry Wolf (1983), Score: 4.431767139479905
6. MovieID: 3363, Title: American Graffiti (1973), Score: 4.431767139479905
7. MovieID: 3972, Title: Legend of Drunken Master, The (Jui kuen II) (1994), Score: 4.425417933130698
8. MovieID: 471, Title: Hudsucker Proxy, The (1994), Score: 4.391311528792653
9. MovieID: 5902, Title: Adaptation (2002), Score: 4.346832712058773
10. MovieID: 318, Title: Shawshank Redemption, The (1994), Score: 4.324722990251327
User 8's satisfaction:  0.8822550976054716
User 36's satisfactio

In [19]:
group = [8,36,44]
sequentialGroupPredictionWithSTD(group,10,3)

Iteration: 1 , alpha = 0
The top 10 movies recommended for the group [8, 36, 44] are:
1. MovieID: 177593, Title: Three Billboards Outside Ebbing, Missouri (2017), Score: 4.563391071103838
2. MovieID: 174053, Title: Black Mirror: White Christmas (2014), Score: 4.540030783592607
3. MovieID: 1207, Title: To Kill a Mockingbird (1962), Score: 4.451569403617054
4. MovieID: 2359, Title: Waking Ned Devine (a.k.a. Waking Ned) (1998), Score: 4.431767139479905
5. MovieID: 3347, Title: Never Cry Wolf (1983), Score: 4.431767139479905
6. MovieID: 3363, Title: American Graffiti (1973), Score: 4.431767139479905
7. MovieID: 3972, Title: Legend of Drunken Master, The (Jui kuen II) (1994), Score: 4.425417933130698
8. MovieID: 471, Title: Hudsucker Proxy, The (1994), Score: 4.391311528792653
9. MovieID: 5902, Title: Adaptation (2002), Score: 4.346832712058773
10. MovieID: 318, Title: Shawshank Redemption, The (1994), Score: 4.324722990251327
User 8's satisfaction:  0.8822550976054716
User 36's satisfactio

### 3 most dissimilar users

In [20]:
group = [8,591,598]
sequentialGroupPrediction(group,10,3)

Iteration: 1 , alpha = 0
The top 10 movies recommended for the group [8, 591, 598] are:
1. MovieID: 2867, Title: Fright Night (1985), Score: 5.6695429113887625
2. MovieID: 7016, Title: Over the Top (1987), Score: 5.658855959562
3. MovieID: 4673, Title: Tango & Cash (1989), Score: 5.325522626228666
4. MovieID: 39715, Title: American Pie Presents: Band Camp (American Pie 4: Band Camp) (2005), Score: 5.325522626228666
5. MovieID: 50189, Title: American Pie Presents The Naked Mile (American Pie 5: The Naked Mile) (2006), Score: 5.325522626228666
6. MovieID: 1649, Title: Fast, Cheap & Out of Control (1997), Score: 5.22589211773797
7. MovieID: 2166, Title: Return to Paradise (1998), Score: 5.22589211773797
8. MovieID: 2417, Title: Heartburn (1986), Score: 5.22589211773797
9. MovieID: 2469, Title: Peggy Sue Got Married (1986), Score: 5.22589211773797
10. MovieID: 2813, Title: Source, The (1999), Score: 5.22589211773797
User 8's satisfaction:  0.6245449556865683
User 591's satisfaction:  1.0
U

In [21]:
group = [8,591,598]
sequentialGroupPredictionWithSTD(group,10,3)

Iteration: 1 , alpha = 0
The top 10 movies recommended for the group [8, 591, 598] are:
1. MovieID: 2867, Title: Fright Night (1985), Score: 5.6695429113887625
2. MovieID: 7016, Title: Over the Top (1987), Score: 5.658855959562
3. MovieID: 4673, Title: Tango & Cash (1989), Score: 5.325522626228666
4. MovieID: 39715, Title: American Pie Presents: Band Camp (American Pie 4: Band Camp) (2005), Score: 5.325522626228666
5. MovieID: 50189, Title: American Pie Presents The Naked Mile (American Pie 5: The Naked Mile) (2006), Score: 5.325522626228666
6. MovieID: 1649, Title: Fast, Cheap & Out of Control (1997), Score: 5.22589211773797
7. MovieID: 2166, Title: Return to Paradise (1998), Score: 5.22589211773797
8. MovieID: 2417, Title: Heartburn (1986), Score: 5.22589211773797
9. MovieID: 2469, Title: Peggy Sue Got Married (1986), Score: 5.22589211773797
10. MovieID: 2813, Title: Source, The (1999), Score: 5.22589211773797
User 8's satisfaction:  0.6245449556865683
User 591's satisfaction:  1.0
U